In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [0]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1].replace(r'\n', ' ', regex=True),
    'past':''*test_df.shape[1],
    'modal':''*test_df.shape[1],
    'antecedent':''*test_df.shape[1],
    'consequent':''*test_df.shape[1]
})

  
# display 
test_df.head()

In [0]:
!pip install allennlp
!pip install nltk
!pip install tqdm

from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

START AGAIN

In [0]:
def extract_modal(obj):
    arr = list() # full set of constituents
    modal= list() # only the modal phrases
    past = list() # only the past phrases
    noun = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr)
                  
                elif k=='nodeType' and v=='NP':
                    arr.append(v)
                elif k=='nodeType' and v=='MD':
                    arr.append(v)
                elif k=='nodeType' and v=='VBD': # Verb, past tense
                    arr.append(v)
                elif k=='nodeType' and v=='VBN': # Verb, past participle
                    arr.append(v)
                elif k=='nodeType' and v=='RB': # Adverb (eg "Even" if past...)  
                    arr.append(v)
                elif k=='nodeType' and v=='IN':  # Preposition or conjunction (eg "if")
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:

              if isinstance(item, dict):
                  if 'MD' in arr: 
                    arr.remove('MD')
                    modal.append(obj)
                    past.append(obj)
                  elif 'NP' in arr: 
                    arr.remove('NP')
                    noun.append(obj)  
                  elif 'VBD' in arr: 
                    arr.remove('VBD')
                    past.append(obj) 
                  elif 'VBD' in arr: 
                    arr.remove('VBN')
                    past.append(obj)
                  elif 'RB' in arr: 
                    arr.remove('RB')
                    if 'even' in obj[0]['word'].lower():
                      past.append(obj)
                  elif 'IN' in arr: 
                     arr.remove('IN')   
                     if 'if' in obj[0]['word'].lower():  
                       past.append(obj)
                      #print(obj)
     
              #arr.append(item) # uncomment for the full set
              #print(item)

              extract(item, arr)
  
        return arr

    results = extract(obj, arr)
    return modal, past, noun # result

In [0]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents():
 modal = list()
 nounm = list()
 past = list() 
 nounp = list()
 nouns = list()

 for key, value in prediction.items():
  if (key=='hierplane_tree'):
    for key, value in value.items():
      if (key=='root'):
        for key, value in value.items():
          if (key == 'children'):

            mod, pas, nou = extract_modal(value)
            
            for u in nou:
              nu = simplify(u) 
              if nu not in nouns:
                nouns.append(nu) 

            for m in mod:
              md = simplify(m) # string gist
              for nm in reversed(sorted(nouns,key=len)):  
                if nm in md:   
                  nounm.append(nm)
                  nouns.remove(nm)

              for p in pas:
                pa = simplify(p)

                if pa in md and len(md) > len(pa):
                   if md.index(pa) > 0 :
                     md = md[0:md.index(pa)]
                   else:
                     md = str(md).replace(str(pa),'')   
                if md in pa and len(pa) > len(md):
                   if pa.index(md) > 0 :
                     pa = pa[0:pa.index(md)]  
                   else:
                     pa = str(pa).replace(str(md),'')  
                for np in reversed(sorted(nouns,key=len)):  
                  if np in pa and len(pa) > len(np): 
                    if pa.index(np) > 0 :
                      pa = pa[0:pa.index(np)]
                    else:
                      pa = str(pa).replace(str(np),'')  
                    nounp.append(np)
                    nouns.remove(np)
                if md in pa:
                   if pa.index(md) > 0 :
                     pa = pa[0:pa.index(md)]  
                   else:
                     pa = str(pa).replace(str(md),'')      

                if pa not in past:
                  past.append(pa) 
              if md not in modal:        
                modal.append(md) 
                
              for p1 in past:
                for p2 in past:
                  if p2 in p1:
                    past.remove(p2) 
              
            return modal, past, nouns, nounm, nounp

In [0]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  gist = list();
  sentence = set()  # rebuild with checking for overlap
  for c in constit:
      #print(c)
      p = separate_children(c) # candidate phrases
      #print(p)
      for head in list(p):
        #print(head)
        phrase = word_tokenize(head)
        #print(phrase)
        words = set(phrase)
        if len(sentence.intersection(words)) < len(words):
          sentence.update(words)
          gist.extend(phrase)
  #print(gist)
  return ' '.join(map(str, gist))

In [147]:
import nltk
nltk.download('punkt')

text = "67% of the embryos exposed to CRISPR-cas9 had only the correct version of the gene - higher than the 50% that would have been expected had the technique not been used."
text.replace('\"','\'')
#print(text)

prediction = predictor.predict(sentence=text)
modal, past, nouns, nounm, nounp = extract_constituents()
print(nounm)
print(modal)
print(past)
print(nounp)

mindex = int(0)
pindex = int(0)
length = int(0)

try : 
  for m in modal:
    if len(m) > mindex and m in text :
      pindex = text.index(m)
      length = len(m)

    for n in nounp:
      if mindex > text.index(n):
        mindex = text.index(n)  

  print(text[mindex:pindex+length])

  mindex = int(0)
  pindex = int(0)
  length = int(0)

  for p in past:
    if len(p) > pindex and p in text:
      pindex = text.index(p)
      mindex = text.index(p)
      length = len(p) 

    for n in nounm:
      if mindex > text.index(n):
        mindex = text.index(n)
      
  print(text[mindex:pindex+length])    
except ValueError:
  print(mindex,pindex,length)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['the technique']
['would have been expected ']
['had the technique not been used']
['only the correct version of the gene']
67% of the embryos exposed to CRISPR-cas9 had only the correct version of the gene - higher than the 50% that would have been expected 
had the technique not been used


In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1].replace(r'\n', ' ', regex=True),
    'past':''*test_df.shape[1],
    'modal':''*test_df.shape[1],
    'antecedent':''*test_df.shape[1],
    'consequent':''*test_df.shape[1]
})

count = test_df['id'].count()
print(count)

for ind in tqdm(test_df.index): 
  prediction = predictor.predict(sentence=test_df['text'][ind])
  modal, past, nouns, nounm, nounp = extract_constituents()
  test_df['modal'][ind] = modal
  test_df['past'][ind] = past

  text = test_df['text'][ind]
  mindex = int(0)
  pindex = int(0)
  length = int(0)

  try : 
    for m in modal:
      if len(m) > mindex and m in text :
        mindex = text.index(m)
        length = len(m)

    test_df['consequent'][ind]=text[mindex:mindex+length]

    mindex = int(0)
    pindex = int(0)
    length = int(0)

    for p in past:
      if len(p) > pindex and p in text:
        pindex = text.index(p)
        mindex = text.index(p)
        length = len(p) 

      for n in nounp:
        if mindex > text.index(n):
          mindex = text.index(n)
        
    test_df['antecedent'][ind] = text[mindex:pindex+length]    
  except ValueError:
    print(mindex,pindex,length)

  sleep(1)

test_df.to_csv(prefix+'test.tsv', sep='\t', index=False, header=False)


In [0]:
!mv /content/test.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/